In [1]:
# %load PlannerDataTranslation.py
import autogen
import json

import pprint
# construct PrettyPrinter first
pp = pprint.PrettyPrinter(indent=4, width=80, compact=False)



In [2]:

config_list = [
    {
        "model": "llama3", #the name of your running model
        "base_url": "http://52.56.167.144:11434/v1", #the local address of the api
        "api_type": "open_ai",
        "api_key": "ollama", # just a placeholder
    }
]


# set a "universal" config for the agents
agent_config = {
    "seed": 42,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list,
}

# humans
user_proxy = autogen.UserProxyAgent(
   name="Admin",
   human_input_mode="ALWAYS",
   is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
   system_message="You are the Admin. Your role is to provide human feedback and give final approval to the final response.",
   code_execution_config=False,
)


planner = autogen.AssistantAgent(
    name="Planner",
    system_message='''You are the Planner. 
    1-) create translation rules for the source data model to target data model.
    2-) create a list of actions accordingly.
    3- create a python function called translator_function

Submit your plan to the Reviewer. If approved, you will respond with a 'TERMINATE' response. 
If rejected , review the feedback, make necessary amendments, and resubmit with 'PLEASE REVIEW' to the reviewer''',
    llm_config=agent_config,
)

reviever = autogen.AssistantAgent(
    name="Reviever",
    system_message='''You are the Reviewer. 
    Your task is to review the Planner's suggested plan translation rules to ensure they meet the required criteria. 
    
    1-)Carefully evaluate the plan.
    2-)Find inconsistencies.
    3-)Provide suggestions.

- If the plan falls short, specify which criteria were not met and provide detailed feedback for revision.

''',
    llm_config=agent_config,
)

user_proxy.description="The Admin. Your role is to provide human feedback and give final approval to the final response."
planner.description="The Planner. Your task is to suggest translation rules from a source data model to a target data model"
reviever.description="The Reviewer. Your task is to review the Planner's suggested translation rules to ensure they meet the required criteria."

# start the "group chat" between agents and humans
groupchat = autogen.GroupChat(agents=[planner, reviever,user_proxy], messages=[], send_introductions=True,max_round=50,speaker_selection_method="round_robin")
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=agent_config,
    system_message='''Initiate the workflow by asking the Planner to suggest translation rules from the source data model to the target data model. The Admin will review and approve the plan.''')

# Start the Chat!
res=manager.initiate_chat(
    planner,
    message="""
The task is to translate from  source data model to target data model. 
An example of source and target data structures are below:
source model example in json format:
    {
        "volume":177.0730798,
        "area":434.4258052,
        "length":0.0,
        "VolumeUnit":"m3",
        "Category":"CV-CV-Abutment-G-P",
        "ss_epd_id":"9cf0bb8930ab4d3a9e8082b475796fae",
        "ss_category_name":"ReadyMix",
        "Embodied_Carbon":54553.4986645559
    }

target model example translated from the example of source data model in json format:
    {
        "Name_notes":"CV-CV-Abutment-G-P",
        "Asset_Code":"9cf0bb8930ab4d3a9e8082b475796fae",
        "Quantity":177.0730798,
        "Unit":"m3",
        "Total":54553.4986645559,
        "Concrete":54553.4986645559
    }

another example:
source model example in json format:
source data model:
    {
        "volume":1.42005247,
        "area":74.7313579,
        "length":0.0,
        "VolumeUnit":"m3",
        "Category":"CV-CV-Barrier-G-P",
        "QTY":110,
        "ss_epd_id":"5e99ea4bf1124b66a0899c4e072550f4",
        "ss_category_name":"Steel",
        "Embodied_Carbon":6780.3888439634
    }
target model example translated from the example of source data model in json format:
    {
        "Name_notes":"CV-CV-Barrier-G-P",
        "Asset_Code":"5e99ea4bf1124b66a0899c4e072550f4",
        "Quantity":1.42005247,
        "Unit":"m3",
        "Total":6780.3888439634,
        "Steel":6780.3888439634,
    }

Here is the source model for you that needs to be tranlated:

    {
        "volume":177.0730798,
        "area":434.4258052,
        "length":0.0,
        "VolumeUnit":"m3",
        "Category":"CV-CV-Abutment-G-P",
        "ss_epd_id":"9cf0bb8930ab4d3a9e8082b475796fae",
        "ss_category_name":"ReadyMix",
        "Embodied_Carbon":54553.4986645559
    }
    
""",
)

# to followup of the previous question, use:
# user_proxy.send(
#     recipient=assistant,
#     message="""your followup response here""",
# )

chat_manager (to Planner):


The task is to translate from  source data model to target data model. 
An example of source and target data structures are below:
source model example in json format:
    {
        "volume":177.0730798,
        "area":434.4258052,
        "length":0.0,
        "VolumeUnit":"m3",
        "Category":"CV-CV-Abutment-G-P",
        "ss_epd_id":"9cf0bb8930ab4d3a9e8082b475796fae",
        "ss_category_name":"ReadyMix",
        "Embodied_Carbon":54553.4986645559
    }

target model example translated from the example of source data model in json format:
    {
        "Name_notes":"CV-CV-Abutment-G-P",
        "Asset_Code":"9cf0bb8930ab4d3a9e8082b475796fae",
        "Quantity":177.0730798,
        "Unit":"m3",
        "Total":54553.4986645559,
        "Concrete":54553.4986645559
    }

another example:
source model example in json format:
source data model:
    {
        "volume":1.42005247,
        "area":74.7313579,
        "length":0.0,
        "VolumeUnit":"m3"

Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [8]:
def translator_function(source_data):
    # Extract values from source data model
    volume = source_data["volume"]
    category = source_data["Category"]
    ss_epd_id = source_data["ss_epd_id"]
    ss_category_name = source_data["ss_category_name"]
    embodied_carbon = source_data["Embodied_Carbon"]

    # Create translated data dictionary
    translated_data = {
        "Name_notes": category,
        "Asset_Code": ss_epd_id,
        "Quantity": volume,
        "Unit": "m3",
        "Total": embodied_carbon,
        "Concrete": embodied_carbon
    }

    return translated_data

In [9]:
    s={
        "volume":177.0730798,
        "area":434.4258052,
        "length":0.0,
        "VolumeUnit":"m3",
        "Category":"CV-CV-Abutment-G-P",
        "ss_epd_id":"9cf0bb8930ab4d3a9e8082b475796fae",
        "ss_category_name":"ReadyMix",
        "Embodied_Carbon":54553.4986645559
    }

    true_t= {
        "Name_notes":"CV-CV-Abutment-G-P",
        "Asset_Code":"9cf0bb8930ab4d3a9e8082b475796fae",
        "Quantity":177.0730798,
        "Unit":"m3",
        "Total":54553.4986645559,
        "Concrete":54553.4986645559
    }

In [10]:
t=translator_function(s)

In [11]:
pp.pprint(t)

{   'Asset_Code': '9cf0bb8930ab4d3a9e8082b475796fae',
    'Concrete': 54553.4986645559,
    'Name_notes': 'CV-CV-Abutment-G-P',
    'Quantity': 177.0730798,
    'Total': 54553.4986645559,
    'Unit': 'm3'}


In [12]:
pp.pprint(true_t)

{   'Asset_Code': '9cf0bb8930ab4d3a9e8082b475796fae',
    'Concrete': 54553.4986645559,
    'Name_notes': 'CV-CV-Abutment-G-P',
    'Quantity': 177.0730798,
    'Total': 54553.4986645559,
    'Unit': 'm3'}
